In [ ]:
import pandas as pd
import os
import glob

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
import numpy as np

from matplotlib import pyplot as plt
import rasterio

In [ ]:
import pprint

In [ ]:
#pprint.pprint(os.listdir('../logs/experiments/runs/'))

In [ ]:
expname = 'sentinel2_patch_swin_04-20_A'

# TODO for topview_swin_04-19_C take test predictions from the runs that were trained on this country
# Same for topview_swin_04-19_D and streetview_swin_04-19_C

In [ ]:
expdir = f'../logs/experiments/runs/{expname}/'
os.listdir(expdir)

In [ ]:
all_logdirs = [os.path.join(expdir, dd) for dd in os.listdir(expdir)]

In [ ]:
logdirs = []
folds = []

fold_key = None

for fold in range(20):
    for logdir in all_logdirs:
        if os.path.exists(os.path.join(logdir, f'test_predictions_fold_{fold}.csv')):
            print(fold, logdir)
            logdirs.append(logdir)
            folds.append(fold)
            
            with open(os.path.join(logdir, 'config_tree.log')) as f:
                config_tree = f.readlines()

In [ ]:
config_tree

In [ ]:
for l in config_tree:
    if 'fold_key' in l:
        fold_key = l.strip().split(' ')[-1]
        break
print(fold_key)

In [ ]:
all_test_df = []
all_valid_df = []

all_pids = []

for fold, logdir in zip(folds, logdirs):
    rnd = {'predicted_labels':'predicted_label'}
    
    tt = pd.read_csv(os.path.join(logdir, f'test_predictions_fold_{fold}.csv')).rename(columns=rnd)
    vv = pd.read_csv(os.path.join(logdir, f'valid_predictions_fold_{fold}.csv')).rename(columns=rnd) 
    dd = pd.read_csv(f'../data/AI4EO-MapYourCity/splits/{fold_key}/split_valid_{fold}.csv')  
    
    tt['fold'] = fold
    vv['fold'] = fold
    
    all_pids.append(tt['pid'].sort_values().values)
    
    print(len(tt['pid']))
    all_test_df.append(tt)
    all_valid_df.append(pd.merge(vv, dd, on='pid'))

In [ ]:
test_df = pd.concat(all_test_df)
test_df['predicted_label'] = test_df['predicted_label'].astype(int)
test_df.pivot_table(index=['pid', 'fold'])

In [ ]:
test_df.assign(experiment=expname).to_csv(f'../submissions/all_folds/{expname}.csv', index=False)

In [ ]:
def get_best_class(sdf):
    '''
    Return the majority vote
    If tied, choose class closest to the mean
    
    '''
    
    spids = []
    slbls = []
    # mode did not work with groupby
    for pid in test_df['pid'].unique():
        dd = test_df.loc[test_df['pid'] == pid]
        mode = dd['predicted_label'].mode()
        mean = dd['predicted_label'].mean()
                
        if len(mode) > 1:
            mode = mode.iloc[np.argmin(np.abs(mode-mean))]
        else:
            mode = mode.values[0]
            
        spids.append(pid)
        slbls.append(mode)
                
    return pd.DataFrame(dict(pid=spids, predicted_label=slbls)).sort_values('pid')

In [ ]:
submission_df = get_best_class(test_df)

In [ ]:
submission_df.to_csv(f'../submissions/{expname}.csv', index=False)
submission_df.head()

In [ ]:
sns.barplot(submission_df['predicted_label'].value_counts());

In [ ]:
reference_submissions = ['streetview_swinv2_trafo.csv',
                         'topview_swinv2_trafo.csv',
                         'merged_topstreet_swin_04-16_C.csv',
                         f'{expname}.csv',
                        ]

In [ ]:
all_submissions = pd.concat([pd.read_csv(os.path.join('../submissions/', f))['predicted_label'].value_counts().to_frame().assign(source=f) \
                             for f in reference_submissions])

In [ ]:
sns.barplot(all_submissions, x='predicted_label', y='count', hue='source');

In [ ]:
assert test_df['pid'].value_counts().std() == 0 # all pids in all folds

In [ ]:
valid_df = pd.concat(all_valid_df)
valid_df.head()

## Accuracy

In [ ]:
Cn = confusion_matrix(valid_df['label'], valid_df['predicted_label'], normalize='true')
C = confusion_matrix(valid_df['label'], valid_df['predicted_label'])

In [ ]:
sns.heatmap(C, annot=True, fmt='.0f');

In [ ]:
acc = accuracy_score(valid_df['label'].values, valid_df['predicted_label'].values)
print(f'Accuracy score: {acc:.4f}')
print(f'MAP:            {np.diag(Cn).mean():.4f}')

In [ ]:
country_ids = valid_df['country_id'].unique()

plot_dfs = []

for country in country_ids:
    country_df = valid_df.loc[valid_df['country_id'] == country]
    cacc = accuracy_score(country_df['label'].values, country_df['predicted_label'].values)
    
    cmat = confusion_matrix(country_df['label'], country_df['predicted_label'], normalize='true').diagonal()
    
    plot_dfs.append(pd.DataFrame(dict(country=country, classes=list(range(7)), cmat=cmat)))
    
    
    print(f'Country ID: {country}, MCA = {cacc:.4f}, count = {len(country_df)}')

In [ ]:
plot_df = pd.concat(plot_dfs)
sns.barplot(plot_df, x='classes', y='cmat', hue='country')
plt.gca().legend(ncol=2)
plt.show()

In [ ]:
folds = valid_df['fold_y'].unique()
for fold in folds:
    country_df = valid_df.loc[valid_df['fold_y'] == fold]
    cacc = accuracy_score(country_df['label'].values, country_df['predicted_label'].values)
    
    print(f'Fold: {fold}, MCA = {cacc:.4f}, count = {len(country_df)}')

## Inspect mis-classified samples

In [ ]:
# Define paths to data

# input_path = "directory with MapYourCity image files"
input_path = "../data/AI4EO-MapYourCity/v1/building-age-dataset/"
train_path = input_path + "train/data/"
test_path = input_path + "test/data/"

In [ ]:
# Choose a building by pid:
tclass = 2
pclass = 3
mismatched_df = valid_df.query('label==@tclass and predicted_label==@pclass')
print(f'{len(mismatched_df)} samples were classified as {pclass} but are {tclass}')

pid = mismatched_df['pid'].iloc[np.random.randint(len(mismatched_df))]
print(valid_df.loc[valid_df['pid'] == pid][['pid', 'country_id', 'city_id']])
street = plt.imread(f"{train_path}{pid}/street.jpg")
orthophoto = plt.imread(f"{train_path}{pid}/orthophoto.tif")
s2 = rasterio.open(f"{train_path}{pid}/s2_l2a.tif").read()
s2 = np.transpose(s2,[1,2,0])

# Show the 3 modalities - street view, orthophoto and Seninel-2

fig, axs = plt.subplots(figsize=(15, 15), nrows=1, ncols = 3)
axs = axs.flatten()
axs[0].imshow(street)
axs[1].imshow(orthophoto)
axs[2].imshow(s2[...,[3,2,1]]*3e-4)

axs[0].set_title("Street")
axs[1].set_title("Orthophoto")
axs[2].set_title("Sentinel-2-L2A")

plt.show()

## Merge streetview

In [ ]:
df1 = pd.read_csv('../submissions/topview_swin_04-19_A.csv').rename(columns={'predicted_label':'topview_label'})
df3 = pd.read_csv('../submissions/sentinel2_patch_swin_04-20_A.csv').rename(columns={'predicted_label':'sentinel2_label'})
df2 = pd.read_csv('../submissions/streetview_swin_04-19_A.csv').rename(columns={'predicted_label':'streetview_label'})
is_valid_streetview = pd.read_csv('valid_streetview.csv')
target = '../submissions/merged_topstreet_swin_04-19_A.csv'

In [ ]:
df2 = df2[is_valid_streetview['is_valid_streetview']]

In [ ]:
df = pd.merge(df3, pd.merge(df1, df2, 'outer'), 'outer')

In [ ]:
predicted_label = []
for i in range(len(df)):
    if np.isnan(df.iloc[i]['streetview_label']) or \
               df.iloc[i]['topview_label'] == df.iloc[i]['streetview_label']:
        predicted_label.append(df.iloc[i]['topview_label'])
    else:
        predicted_label.append(df.iloc[i]['streetview_label'])
        

In [ ]:
df['predicted_label'] = np.array(predicted_label).astype(int)
df = df.drop(columns=['topview_label', 'streetview_label'])
df.to_csv(target, index=False)

In [ ]:
df.head()

In [ ]:
df = df[is_valid_streetview['is_valid_streetview']]

In [ ]:
(df.iloc[:,1:].std(axis=1) == 0).sum()

In [ ]:
dissent = df['topview_label'] != df['streetview_label']

In [ ]:
df = df[dissent]

In [ ]:
s2_vs_top = abs(df['topview_label'] - df['sentinel2_label'])
s2_vs_str = abs(df['streetview_label'] - df['sentinel2_label'])

In [ ]:
tmp = pd.DataFrame(dict(pid=df.pid.values, s2_vs_top=s2_vs_top.values, s2_vs_str=s2_vs_str))
tmpix = np.argmin(tmp.iloc[:,1:].values, axis=1)
pid_top = tmp['pid'][tmpix == 0]
pid_str = tmp['pid'][tmpix == 1]

In [ ]:
len(pid_top), len(pid_str)

## Merge streetview with majority vote

In [ ]:
df1 = pd.read_csv('../submissions/all_folds/topview_swinv2_trafo.csv').rename(columns={'predicted_label':'topview_label'})
df2 = pd.read_csv('../submissions/all_folds/streetview_swinv2_trafo.csv').rename(columns={'predicted_label':'streetview_label'})
target = '../submissions/merged_all-folds_topstreet_swinv2_trafo.csv'

In [ ]:
merged_submission_df = get_best_class(pd.concat([df1, df2]))

In [ ]:
merged_submission_df.to_csv(target, index=False)

In [ ]:
pids = submission_df['pid'].values
tp = '/home/k/k202141/rootgit/AI4EO-MapYourCity/data/AI4EO-MapYourCity/v1/building-age-dataset/test/data/'

In [ ]:
v = [os.path.exists(os.path.join(tp, pid, 'street.jpg')) for pid in pids]

In [ ]:
pd.DataFrame(dict(pid=pids, is_valid_streetview=v)).to_csv('valid_streetview.csv', index=False)

In [ ]:
submission_df.head()